In [4]:
import tensorflow
from tensorflow.keras.models import Model
from tensorflow.keras.layers import Input, LSTM,Dense
import numpy as np




In [5]:
#language transalator 
#franch to english

#data path

data_path="franch.txt"
input_texts=[]
input_character=set()
output_texts=[]
output_character=set()
num_samples=20000
with open(data_path,"r",encoding="utf-8") as f:
    lines=f.read().split("\n")


for line in lines[:num_samples]:
        input_text,output_text,=line.split("\t")
        output_text="\t"+output_text+"\n"
        input_texts.append(input_text)
        output_texts.append(output_text)
        for char in input_text:
            if char not in input_character:
                input_character.add(char)
        for char in output_text:
            if char not in output_character:
                output_character.add(char)


input_character=sorted(list(input_character))
output_character=sorted(list(output_character))
num_encoder_tokens=len(input_character)
num_decoder_tokens=len(output_character)
max_encoder_seq_length=max([len(txt) for txt in input_texts])
max_decoder_seq_length=max([len(txt) for txt in output_texts])



print("Number of samples:",len(input_texts))
print("Number of unique input tokens:",num_encoder_tokens)

print("Number of unique output tokens:",num_decoder_tokens)
print("Max sequence length for inputs:",max_encoder_seq_length)
print("Max sequence length for outputs:",max_decoder_seq_length)

input_token_index=dict([(char,i) for i,char in enumerate(input_character)])
output_token_index=dict([(char,i) for i,char in enumerate(output_character)])




Number of samples: 20000
Number of unique input tokens: 48
Number of unique output tokens: 70
Max sequence length for inputs: 20
Max sequence length for outputs: 60


In [9]:
encoder_input_data=np.zeros((len(input_texts),max_encoder_seq_length,num_encoder_tokens),dtype="float32")
decoder_input_data=np.zeros((len(input_texts),max_decoder_seq_length,num_decoder_tokens),dtype="float32")
decoder_output_data=np.zeros((len(input_texts),max_decoder_seq_length,num_decoder_tokens),dtype="float32")

for i,(input_text,output_text) in enumerate(zip(input_texts,output_texts)):
    for t,char in enumerate(input_text):
        encoder_input_data[i,t,input_token_index[char]]=1.0
    encoder_input_data[i,t+1:,input_token_index[" "]]=1.0
    for t,char in enumerate(output_text):
        decoder_input_data[i,t,output_token_index[char]]=1.0
        if t>0:
            decoder_input_data[i,t-1,output_token_index[char]]=1.0
    decoder_input_data[i,t+1:,output_token_index[" "]]=1.0
    decoder_input_data[i,t:,output_token_index[" "]]=1.0
# print("decoder_input_data::",decoder_input_data)




#encoder
encoder_inputs=Input(shape=(None,num_encoder_tokens))
encoder=LSTM(256,return_state=True)


batch=64
epoch=100
latent_dim=265




encoder_outputs,state_h,state_c=encoder(encoder_inputs)
encoder_states=[state_h,state_c]
#decoder
decoder_inputs=Input(shape=(None,num_decoder_tokens))
decoder_lstm=LSTM(256,return_sequences=True,return_state=True)
decoder_outputs,_,_=decoder_lstm(decoder_inputs,initial_state=encoder_states)
decoder_dense=Dense(num_decoder_tokens,activation="softmax")
decoder_outputs=decoder_dense(decoder_outputs)
model=Model([encoder_inputs,decoder_inputs],decoder_outputs)
model.compile(optimizer="rmsprop",loss="categorical_crossentropy",metrics=["accuracy"])
model.fit([encoder_input_data,decoder_input_data],decoder_input_data,batch_size=64,epochs=50,validation_split=0.2)
model.save("s2s.h5")
# encoder_model=Model(encoder_inputs,encoder_states)
# decoder_state_input_h=Input(shape=(256,))
# decoder_state_input_c=Input(shape=(256,))
# decoder_states_inputs=[decoder_state_input_h,decoder_state_input_c]
# decoder_outputs,state_h,state_c=decoder_lstm(decoder_inputs,initial_state=decoder_states_inputs)
# decoder_states=[state_h,state_c]
# decoder_outputs=decoder_dense(decoder_outputs)
# decoder_model=Model([decoder_inputs]+decoder_states_inputs,[decoder_outputs]+decoder_states)
# reverse_input_char_index=dict((i,char) for char,i in input_token_index.items())
# reverse_output_char_index=dict((i,char) for char,i in output_token_index.items())
# def decode_sequence(input_seq):
#     states_value=encoder_model.predict(input_seq)
#     target_seq=np.zeros((1,1,num_decoder_tokens))
#     target_seq[0,0,output_token_index["\t"]]=1.0
#     stop_condition=False
#     decoded_sentence=""
#     while not stop_condition:
#         output_tokens,h,c=decoder_model.predict([target_seq]+states_value)
#         sampled_token_index=np.argmax(output_tokens[0,-1,:])
#         sampled_char=reverse_output_char_index[sampled_token_index]
#         decoded_sentence+=sampled_char
#         if sampled_char=="\n" or len(decoded_sentence)>max_decoder_seq_length:
#             stop_condition=True
#         target_seq=np.zeros((1,1,num_decoder_tokens))
#         target_seq[0,0,sampled_token_index]=1.0
#         states_value=[h,c]
#     return decoded_sentence
# for seq_index in range(20):
#     input_seq=encoder_input_data[seq_index:seq_index+1]
#     decoded_sentence=decode_sequence(input_seq)
#     print("-")
#     print("Input sentence:",input_texts[seq_index])
#     print("Decoded sentence:",decoded_sentence)



Epoch 1/50
 20/250 ━━━━━━━━━━━━━━━━━━━━ 40s 178ms/step - accuracy: 0.6105 - loss: 3.9994

KeyboardInterrupt: 